In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [21]:
load_dotenv()

llm = ChatGroq(
    api_key=os.environ.get("GROQ_API_KEY"),
    model="openai/gpt-oss-20b"
)

# response = llm.invoke("Who is the president of Sri-Lanka?")
# print(response.content)

In [3]:
#from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# pdf_url = "https://www.gptaiflow.com/assets/files/2025-01-18-pdf-1-TechAI-Goolge-whitepaper_Prompt%20Engineering_v4-af36dcc7a49bb7269a58b1c9b89a8ae1.pdf"

# loader = UnstructuredURLLoader(urls=[pdf_url])
# documents = loader.load()

loader = PyMuPDFLoader("Docs/prompt_engineering.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

print(docs[100].page_content)

Prompt Engineering
September 2024
57
Growing research suggests that focusing on positive instructions in prompting can be more 
effective than relying heavily on constraints. This approach aligns with how humans prefer 
positive instructions over lists of what not to do. 
Instructions directly communicate the desired outcome, whereas constraints might leave the 
model guessing about what is allowed. It gives flexibility and encourages creativity within the 
defined boundaries, while constraints can limit the model’s potential. Also a list of constraints 
can clash with each other.
Constraints are still valuable but in certain situations. To prevent the model from generating 
harmful or biased content or when a strict output format or style is needed.
If possible, use positive instructions: instead of telling the model what not to do, tell it what to 
do instead. This can avoid confusion and improve the accuracy of the output. 
DO:


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(docs, embeddings)

# optional
vector_db.save_local("faiss_index")

print("Embeddings stored in FAISS vector database.")

/var/folders/7l/sh30bz8d77z_s4bg0s58rv6r0000gn/T/ipykernel_14566/3954922425.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/senash/Desktop/Prompt Tutor - Agentic RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings stored in FAISS vector database.


In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import ZeroShotAgent, AgentType, initialize_agent
from tools import retriever_tool, prompt_generator_tool, query_classifier_tool


tools = [retriever_tool, prompt_generator_tool, query_classifier_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)


def query_pipeline(user_query):
    result = agent({"input": user_query, "chat_history": []})
    return result["output"]

user_query = "What are the fundamentals a user should know about prompt engineering?"
print(query_pipeline(user_query))




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No  
AI: Prompt engineering is the art and science of crafting instructions (prompts) that elicit the best possible responses from language models. Below are the core fundamentals a user should master:

| # | Fundamental | What It Means | Why It Matters |
|---|------------|--------------|---------------|
| 1 | **Clarity & Precision** | Use concise, unambiguous language. | Ambiguity leads to inconsistent or irrelevant answers. |
| 2 | **Contextualization** | Provide enough background for the model to understand the task. | Models are stateless; they need explicit context. |
| 3 | **Task Definition** | State the desired outcome (e.g., “Explain in 3 bullet points”). | Guides the model toward the correct format. |
| 4 | **Instruction Hierarchy** | Separate instructions (e.g., “First, list the steps; then explain each”). | Helps the model follow multi-step reasoning. |
| 5 | **Iterative Refinement** | Start with a dra